In [ ]:
import base64
import requests
import os
import time

# OpenAI API Key
api_key = "*************"

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
input_dir = "/kaggle/input/gpt-dataset-final/PPE_Detection 2.v1i.yolov8-obb/test/images"
output_dir = "/kaggle/working/test_set"
os.makedirs(output_dir, exist_ok=True)

for file_name in os.listdir(input_dir):
    filepath = os.path.join(input_dir, file_name)
    base64_image = encode_image(filepath)
    headers = {
      "Content-Type": "application/json",
      "Authorization": f"Bearer {api_key}"
    }
    payload = {
      "model": "gpt-4o",
      "messages": [
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": ("Are there PPEs in the scene? "
                    "Are there blue, orange or yellow high visibility vests in the scene? "
                    "Are there gloves? "
                    "Are there security boots? "
                    "Are there hard hats? "
                    "Count every PPE present in the scene, I'll make you an example:\n"
                    "    - People: {number of people}\n"
                    "    - Hard_hats: {number of Hard_hats}\n"
                    "    - Gloves: {number of Gloves}\n"
                    "    - Reflective_vests: {number of Reflective_vests}\n"
                    "    - Security_boots: {number of Security_boots}\n")
            },
            {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
              }
            }
          ]
        }
      ],
      "max_tokens": 500
    }
    start = time.time()
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    elapsed_time = time.time() - start
    # Extracting and printing only the response content
    response_json = response.json()
    message_content = response_json['choices'][0]['message']['content']
    #print(f'file: {filepath}, message_content:\n {message_content}\n')

    output_file_path = os.path.join(output_dir, f"{os.path.splitext(file_name)[0]}_output.txt")
    with open(output_file_path, "w") as output_file:
        output_file.write(f"Image filename: {file_name}\n{message_content}\n")
        output_file.write(f"Processing time: {elapsed_time:.4f} seconds\n")

In [ ]:
mapping = [
    ("Worker", "People:"),
    ("Vest", "Reflective_vests:"),
    ("Boots", "Security_boots:"),
    ("Gloves", "Gloves:"),
    ("Hat", "Hard_hats:"),
]

def calculate_metrics(ground_truth, predictions, mapping):
    metrics = {"TP": 0, "FP": 0, "FN": 0}
    for class_name, model_key in mapping:
        if model_key is None:
            pred_count = 0  # No prediction for this class
        else:
            pred_count = predictions.get(model_key, 0)
        true_count = ground_truth.get(class_name, 0)

        if pred_count == true_count:
            metrics["TP"] += true_count
        elif true_count > pred_count:
            metrics["TP"] += pred_count
            metrics["FN"] += (true_count - pred_count)
        else:
            metrics["TP"] += true_count
            metrics["FP"] += (pred_count - true_count)

    return metrics

def calculate_metrics_per_DPI(ground_truth, predictions, mapping, dpi):
    metrics = {"TP": 0, "FP": 0, "FN": 0}
    for class_name, model_key in mapping:
        if model_key is None or dpi != model_key.strip(":"):
            continue  # Skip if the current class does not match the DPI we're focusing on

        pred_count = predictions.get(model_key, 0)
        true_count = ground_truth.get(class_name, 0)

        if pred_count == true_count:
            metrics["TP"] += true_count
        elif true_count > pred_count:
            metrics["TP"] += pred_count
            metrics["FN"] += (true_count - pred_count)
        else:
            metrics["TP"] += true_count
            metrics["FP"] += (pred_count - true_count)

    return metrics



def read_ground_truth(file_path):
    ground_truth = {}
    with open(file_path, 'r') as f:
        for line in f:
            elements = line.split()
            class_id = int(elements[0])
            if class_id == 0:
                ground_truth["Gloves"] = ground_truth.get("Gloves", 0) + 1
            elif class_id == 1:
                ground_truth["Hat"] = ground_truth.get("Hat", 0) + 1
            elif class_id == 4:
                ground_truth["Worker"] = ground_truth.get("Worker", 0) + 1
            elif class_id == 2:
                ground_truth["Boots"] = ground_truth.get("Boots", 0) + 1
            elif class_id == 3:
                ground_truth["Vest"] = ground_truth.get("Vest", 0) + 1
    return ground_truth


def extract_json_from_file(file_path):
    result = {}
    found = False
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            line = line.lstrip()
            if line.startswith('-'):
                DPI = ''
                found = True
                words = line.split()
                flag = True
                for word in words:
                    if not flag:
                        break
                    if word == 'People:' or word == 'Gloves:' or word == 'Security_boots:' or word == 'Reflective_vests:' or word == 'Hard_hats:':
                        DPI = word
                        found = True
                    if word.isdigit():
                        result[DPI] = result.get(DPI, 0) + int(word)
                        flag = False
                    if word == 'Yes':
                        result[DPI] = result.get(DPI, 0) + 1
                        flag = False
                    if word == 'No' or word.lower() == 'not':
                        result[DPI] = result.get(DPI, 0)
                        flag = False
        if not found:
            result['People:'] = result.get('People:', 0)
            result['Gloves:'] = result.get('Gloves:', 0)
            result['Security_boots:'] = result.get('Security_boots:', 0)
            result['Reflective_vests:'] = result.get('Reflective_vests:', 0)
            result['Hard_hats:'] = result.get('Hard_hats:', 0)
    #print(f'result: {result}')
    return result



def filter_greater_than_zero(data):
    return {key: value for key, value in data.items() if value > 0}

def calculate_classification_metrics(metrics):
    TP = metrics["TP"]
    FP = metrics["FP"]
    FN = metrics["FN"]

    accuracy = TP / (TP + FP + FN) if (TP + FP + FN) > 0 else 0
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1_score
    }

def save_metrics_to_file(metrics, output_folder, file_name):
    output_path = os.path.join(output_folder, f"{file_name}_metrics.json")
    with open(output_path, 'w') as f:
        json.dump(metrics, f, indent=4)
    #print(f"Metrics saved to {output_path}")

def calculate_final_metrics(all_metrics):
    total_accuracy = 0
    total_precision = 0
    total_recall = 0
    total_f1_score = 0
    n = len(all_metrics)
    # Somma i valori delle metriche per ciascun file
    for metric in all_metrics:
        total_accuracy += metric['accuracy']
        total_precision += metric['precision']
        total_recall += metric['recall']
        total_f1_score += metric['f1_score']


    # Calcola la media per ciascuna metrica
    final_metrics = {
        "final_accuracy": total_accuracy / n,
        "final_precision": total_precision / n,
        "final_recall": total_recall / n,
        "final_f1_score": total_f1_score / n
    }
    return final_metrics

In [ ]:
import json
import os

ground_truth_folder = "/kaggle/input/gpt-dataset-final/PPE_Detection 2.v1i.yolov8-obb/test/labels"
predictions_folder = "/kaggle/working/test_set"
output_folder = "/kaggle/working/metrics_output"
output_folder_final = "/kaggle/working/metrics_output_final"
os.makedirs(output_folder, exist_ok=True)
os.makedirs(output_folder_final, exist_ok=True)
DPI = ['People', 'Hard_hats', 'Gloves', 'Reflective_vests', 'Security_boots']

gt_files = sorted(os.listdir(ground_truth_folder))
pred_files = sorted(os.listdir(predictions_folder))
all_metrics = []
for gt_file, pred_file in zip(gt_files, pred_files):
    base_filename = os.path.splitext(pred_file)[0]
    gt_results = read_ground_truth(os.path.join(ground_truth_folder,gt_file))
    data = extract_json_from_file(os.path.join(predictions_folder, pred_file))
    prediction_results = filter_greater_than_zero(data)
    metrics = calculate_metrics(gt_results, prediction_results, mapping)
    final_metrics_per_image = calculate_classification_metrics(metrics)
    all_metrics.append(final_metrics_per_image)
final = calculate_final_metrics(all_metrics)
print(f'final metric: {final}')
save_metrics_to_file(final, output_folder_final, "final")

#########################################################################################################
final_ppes_metrics = {
    'People': {
        'Accuracy': 0,
        'Precision': 0,
        'Recall': 0,
        'F1-Score': 0
    },
    'Hard_hats': {
        'Accuracy': 0,
        'Precision': 0,
        'Recall': 0,
        'F1-Score': 0
    },
    'Gloves': {
        'Accuracy': 0,
        'Precision': 0,
        'Recall': 0,
        'F1-Score': 0
    },
    'Reflective_vests': {
        'Accuracy': 0,
        'Precision': 0,
        'Recall': 0,
        'F1-Score': 0
    },
    'Security_boots': {
        'Accuracy': 0,
        'Precision': 0,
        'Recall': 0,
        'F1-Score': 0
    },
}
# Second part of the code to calculate per DPI metrics
gt_files = sorted(os.listdir(ground_truth_folder))
pred_files = sorted(os.listdir(predictions_folder))

# Iterate over each DPI type
for dpi in DPI:
    all_metrics_per_dpi = []  # Initialize a list to collect metrics for each DPI
    for gt_file, pred_file in zip(gt_files, pred_files):
        base_filename = os.path.splitext(pred_file)[0]

        # Read ground truth and predictions for the current image
        gt_results = read_ground_truth(os.path.join(ground_truth_folder, gt_file))
        data = extract_json_from_file(os.path.join(predictions_folder, pred_file))

        # Filter predictions with counts greater than zero
        prediction_results = filter_greater_than_zero(data)

        # Calculate per-image metrics for the current DPI
        metrics = calculate_metrics_per_DPI(gt_results, prediction_results, mapping, dpi)
        final_metrics_per_image = calculate_classification_metrics(metrics)
        all_metrics_per_dpi.append(final_metrics_per_image)

    # Calculate final averaged metrics for the current DPI
    final_dpi_metrics = calculate_final_metrics(all_metrics_per_dpi)
    print(f'final metric {dpi}: {final_dpi_metrics}')

    # Update the final PPE metrics dictionary with calculated values
    final_ppes_metrics[dpi]['Accuracy'] = final_dpi_metrics['final_accuracy']
    final_ppes_metrics[dpi]['Precision'] = final_dpi_metrics['final_precision']
    final_ppes_metrics[dpi]['Recall'] = final_dpi_metrics['final_recall']
    final_ppes_metrics[dpi]['F1-Score'] = final_dpi_metrics['final_f1_score']

# Save final PPEs metrics to file
output_file_path = os.path.join(output_folder_final, "final_ppes_metrics.json")
with open(output_file_path, 'w') as f:
    json.dump(final_ppes_metrics, f, indent=4)

final metric: {'final_accuracy': 0.835035749681239, 'final_precision': 0.8877078918102266, 'final_recall': 0.9345083780727107, 'final_f1_score': 0.8973454519978932}
final metric People: {'final_accuracy': 0.9135960075437926, 'final_precision': 0.9370922150985443, 'final_recall': 0.9670101215591723, 'final_f1_score': 0.9415358220425202}
final metric Hard_hats: {'final_accuracy': 0.8458324498225432, 'final_precision': 0.8760295084662173, 'final_recall': 0.8811953464196172, 'final_f1_score': 0.8710365333383813}
final metric Gloves: {'final_accuracy': 0.1248945147679325, 'final_precision': 0.177373417721519, 'final_recall': 0.13422995780590719, 'final_f1_score': 0.14479720349973507}
final metric Reflective_vests: {'final_accuracy': 0.3928941311852704, 'final_precision': 0.3983341552961806, 'final_recall': 0.4122814948764316, 'final_f1_score': 0.4014231910639207}
final metric Security_boots: {'final_accuracy': 0.032775426050742504, 'final_precision': 0.050237341772151896, 'final_recall': 0.